<a href="https://colab.research.google.com/github/hailong09/fake_news_detection_model_finetuning/blob/main/fake_news_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install tensorflow transformers datasets scikit-learn pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.1 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset
import pandas as pd


# Load Fake news dataset
dataset = load_dataset("liar")

# Keep only relevant columns
dataset = dataset.map(lambda example: {"text": example["statement"], "label": 0 if example["label"] == 0 else 1})
dataset = dataset.remove_columns(['statement'])  # Keep only "text" and "label"

print(dataset["train"][0])  # Check sample


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.16k [00:00<?, ?B/s]

liar.py:   0%|          | 0.00/6.41k [00:00<?, ?B/s]

The repository for liar contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/liar.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/10269 [00:00<?, ? examples/s]

...

Generating validation split:   0%|          | 0/1284 [00:00<?, ? examples/s]

Map:   0%|          | 0/10269 [00:00<?, ? examples/s]

Map:   0%|          | 0/1283 [00:00<?, ? examples/s]

Map:   0%|          | 0/1284 [00:00<?, ? examples/s]

{'id': '2635.json', 'label': 0, 'subject': 'abortion', 'speaker': 'dwayne-bohac', 'job_title': 'State representative', 'state_info': 'Texas', 'party_affiliation': 'republican', 'barely_true_counts': 0.0, 'false_counts': 1.0, 'half_true_counts': 0.0, 'mostly_true_counts': 0.0, 'pants_on_fire_counts': 0.0, 'context': 'a mailer', 'text': 'Says the Annies List political group supports third-trimester abortions on demand.'}


In [4]:
from transformers import TFBertForSequenceClassification, BertTokenizer

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)  # 2 classes: Fake/Real

# Tokenize function
def tokenize_func(example):
  return tokenizer(example["text"], padding="max_length", truncation=True)

# Apply tokenization
dataset = dataset.map(tokenize_func, batched=True)




The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/10269 [00:00<?, ? examples/s]

Map:   0%|          | 0/1283 [00:00<?, ? examples/s]

Map:   0%|          | 0/1284 [00:00<?, ? examples/s]

In [5]:
import tensorflow as tf

# Convert Dataset to TF tensors
def convert_to_tf_dataset(split):
  features = {key: tf.convert_to_tensor(dataset[split][key]) for key in ["input_ids", "attention_mask"]}
  labels = tf.convert_to_tensor(dataset[split]["label"])
  return tf.data.Dataset.from_tensor_slices((features, labels)).batch(16)

# Create TensorFlow datasets
train_dataset = convert_to_tf_dataset("train")
valid_dataset = convert_to_tf_dataset("validation")
test_dataset = convert_to_tf_dataset("test")


In [6]:
# Compile model with Adam optimizer & loss function
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)




In [9]:
# Train the model
model.fit(train_dataset, validation_data=valid_dataset, epochs=3)

Epoch 1/3
642/642 [==============================] - 150s 233ms/step - loss: 0.2978 - accuracy: 0.8912 - val_loss: 0.6925 - val_accuracy: 0.7710
Epoch 2/3
642/642 [==============================] - 149s 233ms/step - loss: 0.2078 - accuracy: 0.9295 - val_loss: 0.8134 - val_accuracy: 0.7516
Epoch 3/3
642/642 [==============================] - 149s 233ms/step - loss: 0.1395 - accuracy: 0.9569 - val_loss: 0.7852 - val_accuracy: 0.7858


In [18]:
# Evaluate on test set
loss, accuracy = model.evaluate(test_dataset)
print(f"Test Accuracy: {accuracy:.4f}")

81/81 [==============================] - 7s 82ms/step - loss: 0.8047 - accuracy: 0.7880
Test Accuracy: 0.7880
